In [36]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict

In [37]:
class PrivateState(TypedDict):
    a: int
    b: int


class InputState(TypedDict):
    hello: str


class OutputState(TypedDict):
    bye: str
    

class MegaPrivateState(TypedDict):
    secret: bool


graph_builder = StateGraph(
    PrivateState,
    input_schema=InputState,
    output_schema=OutputState,
)

In [38]:
def node_first(state: InputState) -> InputState:
    print("node_first", state)
    return {
        "hello": "world"
    }


def node_second(state: PrivateState) -> PrivateState:
    print("node_second", state)
    return {
        "a": 1,
    }


def node_third(state: PrivateState) -> PrivateState:
    print("node_third", state)
    return {
        "b": 10,
    }


def node_fourth(state: PrivateState) -> OutputState:
    print("node_fourth", state)
    return {
        "bye": "world"
    }

def node_fifth(state: OutputState) -> MegaPrivateState:
    return {
        "secret": True
    }

def node_sixth(state: MegaPrivateState):
    print("node_sixth", state)

In [39]:
graph_builder.add_node("node_first", node_first)
graph_builder.add_node("node_second", node_second)
graph_builder.add_node("node_third", node_third)
graph_builder.add_node("node_fourth", node_fourth)
graph_builder.add_node("node_fifth", node_fifth)
graph_builder.add_node("node_sixth", node_sixth)

graph_builder.add_edge(START, "node_first")
graph_builder.add_edge("node_first", "node_second")
graph_builder.add_edge("node_second", "node_third")
graph_builder.add_edge("node_third", "node_fourth")
graph_builder.add_edge("node_fourth", "node_fifth")
graph_builder.add_edge("node_fifth", "node_sixth")
graph_builder.add_edge("node_sixth", END)

In [40]:
graph = graph_builder.compile()

# graph

result = graph.invoke({"hello": "world!!!"})

print("result: ", result)

node_first {'hello': 'world!!!'}
node_second {}
node_third {'a': 1}
node_fourth {'a': 1, 'b': 10}
node_sixth {'secret': True}
result:  {'bye': 'world'}
